In [2]:
from ipycanvas import Canvas

In [161]:
import argparse
from typing import Any, List, Callable, Dict, Tuple

from GridWorld_environments import Grid_World
from RL_agents import ValueIterationAgent, QLearningAgent
from IRL_agents import IRL_from_sampled_trajectories

from sklearn.preprocessing import MinMaxScaler

import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm

from copy import deepcopy

### Definitions

#### Show Policy

In [59]:
def get_display_policy(policy: Dict[Tuple[int], Tuple[int]], actions_to_str_map: Dict[Tuple[int], str], size: Tuple[int], terminal_states: Tuple[int] ):

    policy_str = {state: actions_to_str_map[action] for state, action in policy.items()}
    policy_arr = np.zeros(size, str)

    for state in policy_str:
        policy_arr[state] = policy_str[state]

    for state in terminal_states:
        policy_arr[state] = "x"

    return policy_arr

#### Show Value function

In [173]:
def get_display_value_func(val_func: Dict[Tuple[int], Tuple[int]], size: Tuple[int], terminal_states: Tuple[int] ):

    val_func_disp = np.zeros(size)
    #policy_str = {state: actions_to_str_map[action] for state, action in policy.items()}
    #policy_arr = np.zeros(size, str)

    for state in val_func:
        val_func_disp[state] = val_func[state]

    for state in terminal_states:
        val_func_disp[state] = 0

    return val_func_disp

#### Perform Value evaluation function

In [99]:
def perform_value_evaluation(gw_env: Grid_World, policy: Dict[Any, Any], verbose=False):
    
    vi_agent = ValueIterationAgent(states=gw_env.get_state_space(),
                                   terminal_states=gw_env.get_terminal_states(),
                                   reward_function=gw_env.get_reward_func(),
                                   actions=gw_env.get_action_space(),
                                   gamma=GAMMA)
    
    while not vi_agent.value_converged:
        
        for state in gw_env.get_state_space():
            
            if state in gw_env.get_terminal_states():
                continue
                
            policy_act = policy[state]
            next_state = gw_env.get_new_state_on_action(old_state=state, action=policy_act)
            next_state_value = vi_agent.get_state_value(state=next_state)
            
            vi_agent.set_state_value(state=state, new_value=(gw_env.get_state_reward(state=next_state) + GAMMA * next_state_value))
        
    if verbose:
        gw_env.display_value_function(value_func=vi_agent.get_value_function())

    return vi_agent.get_value_function()
            

#### Perform Action-Value evaluation function (Q-Learning evaluation)

In [100]:
def perform_q_function_evaluation(gw_env: Grid_World, policy: Dict[Any, Any], rounding: int = 3):

    value_func = perform_value_evaluation(gw_env, policy)
    rewards = gw_env.get_reward_func()

    q_function = {}
    for state in gw_env.get_state_space():

        action_values = {}
        for action in gw_env.get_action_space():

            next_state = gw_env.get_new_state_on_action(old_state=state, action=action)
            action_values[action] = np.round(rewards[next_state] + GAMMA * value_func[next_state], rounding)
            
        q_function[state] = action_values

    return q_function
    

#### IRL Reward estimation function

In [5]:
def irl_reward_estimation(env: Grid_World, optimal_trajectories: List[List[Any]], train_func: Callable):

    # store reference reward function
    reward_func_ref = deepcopy(env.get_board())
    print('Reference reward function:\n', reward_func_ref)

    irl_agent = IRL_from_sampled_trajectories(d=(GW_SIZE[0] * 4, GW_SIZE[1] * 4),
                                              env_ranges=((0, GW_SIZE[0]), (0, GW_SIZE[1])),
                                              env_discrete_size=GW_SIZE,
                                              penalty_factor=2,
                                              gamma=GAMMA)

    # step 2: given optimal trajectories, compute the value estimate
    print("Computing value estimates for optimal trajectories...")
    optimal_value_estimate = irl_agent.compute_value_estimate(trajs=optimal_trajectories)

    candidate_policies = [env.construct_random_policy()]
    candidate_value_estimates = []
    reward_func_estimates = []

    # while True:
    for i in range(IRL_TRAINING_N):
        print(f"Iteration {i}...")

        # step 3: generate trajectories and compute the value estimate for a random policy
        print("Generating trajectories for the candidate policy...")
        candidate_trajectories = env.generate_trajectories(policy=candidate_policies[-1],
                                                           n_traj=NUMBER_OF_TRAJECTORIES,
                                                           max_traj_length=MAXIMUM_TRAJECTORY_LENGTH)
        print("Computing value estimates for condidate trajectories...")
        candidate_value_estimates.append(irl_agent.compute_value_estimate(trajs=candidate_trajectories))

        # step 4: obtain new alphas
        print("Solving linear programming...")
        irl_agent.solve_lp(optimal_value_estimate, candidate_value_estimates)

        # step 5: construct new reward function from the alphas
        reward_func = irl_agent.construct_reward_function(alphas=irl_agent.get_alphas())

        # step 6: find optimal policy under new reward function and add to 'candidate_policies' list
        env.set_reward_func(reward_func)
        candidate_policies.append(train_func(gw_env=env, verbose=True))  # train_value_iteration(gw_env=env))
        # store new reward function
        reward_func_estimates.append(env.get_board())
        
        print("Latest estimated reward function:\n", reward_func_estimates[-1])
        env.display_policy(policy=candidate_policies[-1])
        print("============================================================\n" * 2)

    return {'reference_reward_func': reward_func_ref, 'policy_pred': np.mean(np.array([list(pol.values()) for pol in candidate_policies]), axis=0), 'avg_predicted_reward_func': np.mean(np.array(reward_func_estimates), axis=0)}


def calc_value_distance(value_estimates_ref, value_estimates_pred):
    return np.linalg.norm(np.array(value_estimates_ref) - np.array(value_estimates_pred))

# Bayesian Inverse Reinforcement Learning

## Agenda

1. Motivation
2. Reference work
    1. Inverse Reinforcement Learning (IRL)
    2. Bayesian Inverse Reinforcement Learning (BIRL)
3. Implementation
4. Results
5. Outlook
    - Add Priors
        - Gaussian, ...
    - Add different Policies
        - Boltzman
        - Epsilon Greedy
        - Greedy

## 1. Motivation

The problem of Inverse Reinforcement Learning has been defined in Learning agents for uncertain environments (Russel, 1998).

- **Determine**: reward function being optimized.
- **Given**: 1) Measurements of an agent's behavior over time, in a variety of circumstances 2) Measurements of the sensory inputs to that agent; 3) a model of the environment. ( Ng and Russel, 2000 )

### The Bee hive
<img src="presentation/Bee_Hive_IRL.png" alt="bee hive" width="100%"/>
source: Niklas Kaspareit

The problem here: How do we actually properly set rewards for different attributes?

motivation for this problem arises from e.g. animal and human learning.
example: 
Bee foraging -> 
bee searches for nectar. 
reward -> nectar in flower
possibly multiattribute: weight nectar ingestion against flight distance, time, risk from wind and predators.

Conclusion: hard to determine relative weights here

#### Second idea: 
Often we want to learn a policy based on observation of an expert. However it is assumed that the reward function is known, which often is not the case. So we take the policy from the expert as given and infer the reward function.

### Our goal
Compare different IRL algorithms. To be precise compare the IRL algorithm provided by Ng and Russel in 2000 to the Bayesian Inverse Reinforcement Learning algorithm provided by Ramachandran et al. in 2007.
Ramachandran et al. showed a diagram which proved that the BIRL algorithm performs better than the IRL algorithm by Ng and Russel comparing the reward loss and the policy loss.

<br />
<br />


<div style="display: flex;   flex: 33.33%; padding: 1px;">
    <img src="presentation/BIRL_Paper_Reward_Loss.jpg" width=50%/>
    <img src="presentation/BIRL_Paper_Policy_Loss.jpg" width=50%/>
</div>

### Hyper parameters

In [6]:
GAMMA = 0.95
VALUE_ITERATION_TRAINING_N = 25
IRL_TRAINING_N = 10

NUMBER_OF_TRAJECTORIES = 400
MAXIMUM_TRAJECTORY_LENGTH = 10

GW_SIZE = (6, 6)
GW_SIZES = [(3, 3)]  # [(x, x) for x in np.arange(5,11, 5)]
GW_TRAPS = [(3,3)]
GW_GOALS = [(0, 0)]

In [7]:
#environment = Grid_World(size=GW_SIZE, traps=GW_TRAPS, goals=GW_GOALS, randomize_board=False)

In [8]:
#vi_greedy_policy = train_value_iteration(gw_env=environment, verbose=True)

## 1. Inverse Reinforcement Learning (IRL)

Given 
- finite State space $\mathcal{S}$
- a set of $k$ actions $A = \{a_1, \cdots , a_k\}$
- transition probabilities $\{P_{sa}\}$
- discount factor $\gamma$ 
- policy $\pi$ in Markov Decision Process $(S, A, \{P_{sa}\}, \gamma, R)$

Constraint:
- only finite-state MDPs

Required "tools"
- value iteration
- q learning

### Linear function approximation

- For a general solution of $R$ we require all function to map $\mathbb{R}^n \rightarrow \mathbb{R}$
    - Hard to work with this, therefore we choose a linear approximation
    
$ \quad R(s) = \alpha_1 \phi_1(s) + \alpha_2 \phi_2(s) + \cdots + \alpha_d \phi_d(s) $

with the $\phi$s being fixed basis functions mapping from state $S$ into $R$ and the $\alpha_i$s a are the parameters that can be fit and optimized for the reward function.

The idea:
- sample trajectories
<img src="presentation/IRL_trajectories.png" width="50%" height="50%"/>

Using linearity of expectations value function $V_i^\pi$ for the policy $\pi$ is given by

$ \quad V^\pi = \alpha_1 V_1^\pi + \cdots + \alpha_d V_d^\pi$

### IRL from Sampled Trajectories

- Access to policy $\pi$ only given by a set of trajectories.
- Assume that optimal policy can be found by choosing a reward function
- No explicit model of MDP required


<div style="display: flex;   flex: 33.33%; padding: 1px; align-items: center; justify-content: center;">
    <ul style="line-height: normal;">
        <li style="font-size:1.5em;">sample $m$ (optimal) Monte Carlo trajectories under $\pi$</li>
    </ul>
    <img src="presentation/IRL_Algo_Sampled_Trajectories_initial.png" width=65% />
</div>


<div style="display: flex;   flex: 33.33%; padding: 1px; align-items: center; justify-content: center;">
    <ul style="line-height: normal;">
        <li style="font-size:1.5em">
            For each $i = 1, \cdots , d$ define $V_i^\pi(s_0)$.</li>
        <li style="font-size:1.5em; margin: 1em 0em; list-style-type:none;">
            <ul>
                <li style="font-size:0.85em; margin: 0.5em 0em;">Being the average empirical return on the samples trajectories with reward $R$</li>
                <li style="font-size:0.85em; margin: 0.5em 0em;">For any setting of $\alpha_i$s the natural estimate of $\hat{V}^\pi(s_0)$ is:</li>
                <li style="font-size:0.85em; margin: 0.5em 0em;">$\hat{V}^\pi(s_0) = \alpha_1 \hat{V}_1^\pi(s_0) + \cdots + \alpha_d \hat{V}_d^\pi(s_0) $</li>
                <li style="font-size:0.85em; margin: 0.5em 0em;">With: $\hat{V}_i^\pi(s_0) = \phi_i(s_0) + \gamma \phi_i(s_1) + \gamma^2 \phi_i(s_2) + \cdots $ </li>
            </ul>
        </li>
    </ul>
    <img src="presentation/IRL_Algo_Sampled_Trajectories_1.png" width=65% />
</div>


<div style="display: flex;   flex: 33.33%; padding: 1px; align-items: center; justify-content: center;">
    <img src="presentation/IRL_Algo_Sampled_Trajectories_2.png" width=80% />
</div>


execute $m$ Monte Carlo trajectories under $\pi$

Then for each $i = 1, \cdots , d$ define $V_i^\pi(s_0)$ to be what the average empirical return would have been on these on these $m$ trajectories if the reward $R$ had been $\phi_i$.

For any setting of $\alpha_i$s the natural estimate of $\hat{V}^\pi(s_0)$ is:

$ \quad \hat{V}^\pi(s_0) = \alpha_1 \hat{V}_1^\pi(s_0) + \cdots + \alpha_d \hat{V}_d^\pi(s_0) $

With:

$ \quad \hat{V}_i^\pi(s_0) = \phi_i(s_0) + \gamma \phi_i(s_1) + \gamma^2 \phi_i(s_2) + \cdots $ 

So first we determine the optimal policy $\pi^*$ we are given and its value function.
Then we do the same for our randomly generated initial policy $\pi_1$.
Now the idea is to use some "inductive step" to fit the $\alpha_i$s and we want to achieve this:

$ \quad V^{\pi^*} \geq V^{\pi_i}(s_0), \qquad i = 1, \cdots , k$

We have to modify this slightly to get the following optimization:

$ \quad \text{maximize} \sum_{i=1}^{k} p \left ( \hat{V}^{\pi^*}(s_0) - \hat{V}^{\pi_i}(s_0) \right ) $

$ \quad s.t. |\alpha_i| \leq 1, \qquad i = 1, \cdots , d $

Both $\hat{V}^{\pi^*}(s_0)$ and $\hat{V}^{\pi_i}(s_0)$ are linear function. Hence we can easily solve this via linear programming. We use the simplex method.
As described in the paper we choose $p(x) = x$ if $x \geq 0$ and $p(x) = 2x$ if $x < 0$. These values were determined heuristically.

With this we get a new reward function $R = \alpha_1 \phi_1 + \cdots + \alpha_d \phi_d$.
Generate the new policy $\pi_{k+1}$, that maximises $V^{\pi}(s_0)$ with the new reward function and we add the policy to the set of policies.

<div style="display: flex;   flex: 33.33%; padding: 1px;">
    <p>some tet</p>
    <img src="presentation/IRL_Algo_Sampled_Trajectories.jpg" width=65% />
</div>

### Hands On - Experiment

#### Train Value Iteration function

In [132]:
def train_value_iteration(gw_env: Grid_World, verbose=False):
    vi_agent = ValueIterationAgent(states=gw_env.get_state_space(),
                                   terminal_states=gw_env.get_terminal_states(),
                                   reward_function=gw_env.get_reward_func(),
                                   actions=gw_env.get_action_space(),
                                   gamma=GAMMA)

    iters = 0
    while iters < VALUE_ITERATION_TRAINING_N and not vi_agent.value_converged:

        for state in gw_env.get_state_space():

            if state in gw_env.get_terminal_states():
                continue

            opt_act = vi_agent.get_optimal_action(action_state_pairs=gw_env.get_action_state_pairs(state=state))
            next_state = gw_env.get_new_state_on_action(old_state=state, action=opt_act)
            next_state_value = vi_agent.get_state_value(state=next_state)

            vi_agent.set_state_value(state=state, new_value=(gw_env.get_state_reward(state=next_state) + GAMMA * next_state_value))

        iters += 1

    if verbose:
        gw_env.display_value_function(value_func=vi_agent.get_value_function())

    vi_agent.construct_greedy_policy(gw_env.get_action_state_pairs)

    if verbose:
        gw_env.display_policy(policy=vi_agent.get_policy())

    return vi_agent.get_policy()


#### Train Q Learning function

In [133]:
def train_q_learning(gw_env: Grid_World, n_episodes=5000, verbose=False, policy="eps_greedy", eps=0.2, max_episode_len=100, gamma=0.95):
    ql_agent = QLearningAgent(states=gw_env.get_state_space(),
                              size=gw_env.get_board_shape(),
                              terminal_states=gw_env.get_terminal_states(),
                              reward_function=gw_env.get_reward_func(),
                              actions=gw_env.get_action_space(),
                              gamma=gamma)
    
    # init episodes
    episodes = []
    
    # Define state_space without terminal states for getting starting position
    state_space = deepcopy(gw_env.get_state_space()) # all states
    
    board_size = gw_env.get_board_shape()
    total_states = board_size[0] * board_size[1]
    
    # Number 15 is empirically determined.
    # For a 3x3 Grid the total states are 9 and we checked, that at least 100 states are required to produce reasonably reliable results
    # So 9 * x >= 100 yields that x >= 10
    # Now we also added a buffer and therefore chose 15
    convergence_criterion = total_states * 30
    
    terminal_states = gw_env.get_terminal_states()
    for terminal_state in terminal_states:
        state_space.remove(terminal_state) # not non absorbing state_space
    
    # init state_visited_counter
    state_visited = {state: 4 for state in state_space}
    
    #action_value_converged = False
    convergence_counter = 0
    
    for n in range(n_episodes):
        
        episode = []
        
        # reset if every state has been visited at least 4 times (for each action)
        if ( (np.array(list(state_visited.values())) <= 0).all() ):
            state_visited = {state: 4 for state in state_space}
        
        # random starting position
        states_not_visited = [ state for state in state_visited if state_visited[state] > 0 ]
        if len(states_not_visited) > 0:
            start_idx = (np.random.choice(len(states_not_visited)))
            start = states_not_visited[start_idx]
        else:
            start_idx = (np.random.choice(len(state_space)))
            start = state_space[start_idx]
        
        state_visited[start] -= 1
        
        episode.append(start)
        
        i = 0
        terminal = False
        
        old_q_val_func = ql_agent.get_Q_function(mat_repr=True)
        
        while ( ( i < max_episode_len ) and ( not terminal ) ):
            i += 1
            
            # Choose Action from S derived by given policy
            if policy == "eps_greedy":
                if np.random.uniform() < (1-eps):
                    # Choose greedy action -> highest Q-Value
                    chosen_action = ql_agent.get_greedy_action(episode[-1])
                else:
                    # Choose random action form action space
                    action_space = gw_env.get_action_space()
                    chosen_action = action_space[np.random.choice(len(action_space))]
            
            new_state = gw_env.get_new_state_on_action(episode[-1], chosen_action)
            
            # Reward is taken from Q_learning agent -> it knows the reward function from the environment
            ql_agent.update_Q_value(episode[-1], new_state, chosen_action)
            
            episode.append(new_state)
            
            if new_state in terminal_states:
                terminal = True
            else:
                # add to state visited counter for the new state if it is not terminal
                state_visited[new_state] -= 1
                #if (state_visited[new_state] >= 5):
                    #state_visited[new_state] = 0
                    
        episodes.append(episode)
                    
        # essentially works nicely, but to be used carefully. States that will not be visited by the current policy
        # will only be visited, when the start is chosen by random choice in this state
        # -> Fixed by: rarely visited states will be preferred for the choice of the start
        
        # Check if Q-function did is close to the Q-function from the last episode
        if np.isclose( old_q_val_func, ql_agent.get_Q_function(mat_repr=True), atol=1e-08 ).all( ):
            convergence_counter += 1
            
            # Comment in print statements to see how the episodes develop until convergence
            #print("--------------")
            #print(f"episode {n}")
            #print("convergence_counter", convergence_counter)
            #print("--------------")
            
            if convergence_counter >= convergence_criterion:
                break
        else:
            convergence_counter = 0

        
        
    if verbose:
        if n < n_episodes:
            print(f"It took {n} episodes to converge to the optimal Q-function")
        else:
            print(f"Did not converge to optimal Q-function in {n_episodes} episodes")
    
    if verbose:
        gw_env.display_q_function(q_func=ql_agent.get_Q_function())

    ql_agent.construct_greedy_policy(gw_env.get_action_state_pairs)

    if verbose:
        gw_env.display_policy(policy=ql_agent.get_policy())

    return ql_agent.get_policy()       


In [134]:
actions_to_str_map = {(1, 0): "v", (0, 1): ">", (-1, 0): "^", (0, -1): "<", None: "-"}

GAMMA = 0.95
VALUE_ITERATION_TRAINING_N = 100
IRL_TRAINING_N = 2

GW_SIZE = (3, 3)
GW_TRAPS = []
GW_GOALS = [(0, 2)] 

NUMBER_OF_STATES = GW_SIZE[0] * GW_SIZE[1]

NUMBER_OF_TRAJECTORIES = NUMBER_OF_STATES * 20
MAXIMUM_TRAJECTORY_LENGTH = NUMBER_OF_STATES * 4



In [135]:
environment = Grid_World(size=GW_SIZE, traps=GW_TRAPS, goals=GW_GOALS, randomize_board=False)

In [137]:
vi_greedy_policy = train_value_iteration(gw_env=environment, verbose=True)

Value function:
[[0.95     1.       0.      ]
 [0.9025   0.95     1.      ]
 [0.857375 0.9025   0.95    ]]
Policy:
[['>' '>' 'x']
 ['>' '>' '^']
 ['>' '>' '^']]


In [138]:
ql_greedy_policy = train_q_learning(gw_env=environment, verbose=True, policy="eps_greedy", eps=0.2, gamma=GAMMA)

It took 502 episodes to converge to the optimal Q-function
Q function:
[[[0.857375   0.95       0.9025     0.9025    ]
  [0.9025     1.         0.95       0.9025    ]
  [0.         0.         0.         0.        ]]

 [[0.81450625 0.9025     0.9025     0.857375  ]
  [0.857375   0.95       0.95       0.857375  ]
  [0.9025     0.95       1.         0.9025    ]]

 [[0.81450625 0.857375   0.857375   0.81450625]
  [0.857375   0.9025     0.9025     0.81450625]
  [0.9025     0.9025     0.95       0.857375  ]]]
Policy:
[['>' '>' 'x']
 ['>' '>' '^']
 ['>' '>' '^']]


In [139]:
print(f"Generating {NUMBER_OF_TRAJECTORIES} trajectories...")
print("____________________________")
print()

greedy_policy = vi_greedy_policy
# greedy_policy = ql_greedy_policy

trajectories = environment.generate_trajectories(policy=greedy_policy,
                                                 n_traj=NUMBER_OF_TRAJECTORIES,
                                                 max_traj_length=MAXIMUM_TRAJECTORY_LENGTH)

Generating 180 trajectories...
____________________________



In [140]:
print("showing a few generated sample trajectories")
print() 

for i in range(3):
    print(f"random trajectory {i}")
    print()
    example_traj = np.zeros(GW_SIZE, str)
    example_traj[:] = ' '

    for state in trajectories[i]:
        action_char = actions_to_str_map[greedy_policy[state]]
        example_traj[state] = action_char

    for goal in GW_GOALS:
        example_traj[goal] = 'x'


    print(example_traj)
    if i < 2:
        print()
        print("-------------")
        print()

showing a few generated sample trajectories

random trajectory 0

[[' ' ' ' 'x']
 ['>' '>' '^']
 [' ' ' ' ' ']]

-------------

random trajectory 1

[[' ' ' ' 'x']
 [' ' '>' '^']
 [' ' ' ' ' ']]

-------------

random trajectory 2

[[' ' ' ' 'x']
 [' ' ' ' '^']
 [' ' ' ' ' ']]


In [141]:
print("Initializing IRL from samples...")
print()
# restart the environment
environment = Grid_World(size=GW_SIZE, traps=GW_TRAPS, goals=GW_GOALS, randomize_board=False)

train_func = train_value_iteration
# train_func = train_q_learning

#estimated_rewards = irl_reward_estimation(env=environment, optimal_trajectories=trajectories, train_func=train_func)

# ----
#def irl_reward_estimation(env: Grid_World, optimal_trajectories: List[List[Any]], train_func: Callable):

# store reference reward function
reward_func_ref = deepcopy(environment.get_board())
print('Reference reward function:\n', reward_func_ref)

irl_agent = IRL_from_sampled_trajectories(d=(GW_SIZE[0] * 4, GW_SIZE[1] * 4),
                                          env_ranges=((0, GW_SIZE[0]), (0, GW_SIZE[1])),
                                          env_discrete_size=GW_SIZE,
                                          penalty_factor=2,
                                          gamma=GAMMA)



Initializing IRL from samples...

Reference reward function:
 [[0. 0. 1.]
 [0. 0. 0.]
 [0. 0. 0.]]


In [142]:
# step 2: given optimal trajectories, compute the value estimate
print("Computing value estimates for optimal trajectories...")
optimal_value_estimate = irl_agent.compute_value_estimate(trajs=trajectories)

#print()
#print(optimal_value_estimate)



Computing value estimates for optimal trajectories...


In [143]:
candidate_policies = [environment.construct_random_policy()]

print("Starting with the randomly generated initial policy\n")
print(get_display_policy(policy = candidate_policies[-1], actions_to_str_map=actions_to_str_map, size=GW_SIZE, terminal_states=GW_GOALS))


Starting with the randomly generated initial policy

[['^' '<' 'x']
 ['<' 'v' '>']
 ['^' '<' '^']]


In [144]:
len([(1,2),(3,4),(5,4)])

3

In [84]:
# Just for backup to show compute_value_estimate in case someone is interested.
# Cell will be skipped

from timeit import default_timer as timer
from scipy.stats import multivariate_normal

def _approx_func(x, center):
    return multivariate_normal.pdf(x=x, mean=center, cov=5)

def compute_value_estimate(d_centers, trajs: List[List[Any]], gamma = 0.95) -> List[float]:
    """Given a List of trajectories, return a List with the value estimates for each approximating function."""
    
    start = timer()
    value_estimates = []
    np.zeros(len(d_centers) * )
    for approx_center in d_centers:
        temp_value = 0

        for traj in trajs:
            for i, state in enumerate(traj):
                temp_value += gamma**i * _approx_func(x=state, center=approx_center)

        temp_value /= len(trajs)
        value_estimates.append(temp_value)

    end = timer()
    print(f"total time {end - start}")

    return value_estimates


d_rows = 3 * 4
d_cols = 3 * 4
env_cols_min = 3
env_cols_max = 3

# determine the approximating funcs centers
d_row_centers, step = np.linspace(3 - 1, 3 + 1, d_rows, endpoint=False, retstep=True)
d_row_centers += step / 2
# d_row_centers = np.append(d_row_centers, [min(d_row_centers) - step, max(d_row_centers) + step])
# d_row_centers = np.sort(d_row_centers)
d_col_centers, step = np.linspace(env_cols_min - 1, env_cols_max + 1, d_cols, endpoint=False, retstep=True)
d_col_centers += step / 2
# d_col_centers = np.append(d_col_centers, [min(d_col_centers) - step, max(d_col_centers) + step])
# d_col_centers = np.sort(d_col_centers)
# aggregate to one list
d_centers = [(i, j) for i in d_row_centers.round(4) for j in d_col_centers.round(4)]
d_centers

greedy_policy = vi_greedy_policy
# greedy_policy = ql_greedy_policy

trajectoriesTMP = environment.generate_trajectories(policy=greedy_policy,
                                                 n_traj=NUMBER_OF_TRAJECTORIES,
                                                 max_traj_length=MAXIMUM_TRAJECTORY_LENGTH)

compute_value_estimate(d_centers, trajectoriesTMP)

total time 7.191521200000352


[0.06499207379917117,
 0.0638007487017209,
 0.06231172180547912,
 0.06054729838755047,
 0.0585302285994794,
 0.05628949761947457,
 0.05385709909407213,
 0.05126235874931149,
 0.0485400077705921,
 0.045725432814123154,
 0.04284857239204069,
 0.03994343465510973,
 0.06234001678871118,
 0.06118994241009038,
 0.0597548400567997,
 0.058056187778927924,
 0.056115863194973276,
 0.05396171898838458,
 0.05162447457198034,
 0.04913226609404805,
 0.04651840590265634,
 0.04381681817244707,
 0.04105618458591906,
 0.03826908228892398,
 0.05950590108277378,
 0.05840104028125714,
 0.05702461473699593,
 0.0553972057170592,
 0.05353974981034898,
 0.051478881711792754,
 0.04924395429138214,
 0.04686183134322001,
 0.04436430919825518,
 0.04178375197188302,
 0.0391475036209418,
 0.036486618681789484,
 0.056526833004721354,
 0.055470536791219764,
 0.05415675360260484,
 0.05260511033537739,
 0.05083554652699483,
 0.04887340947625175,
 0.046746611181168685,
 0.04448067698641274,
 0.04210580987389455,
 0.03965

In [145]:
candidate_value_estimates = []
reward_func_estimates = []

from timeit import default_timer as timer
start = timer()

# while True:
for i in range(IRL_TRAINING_N):
    print(f"Iteration {i}...")

    # step 3: generate trajectories and compute the value estimate for a random policy
    start = timer()
    print("Generating trajectories for the candidate policy...")
    candidate_trajectories = environment.generate_trajectories(policy=candidate_policies[-1],
                                                       n_traj=NUMBER_OF_TRAJECTORIES,
                                                       max_traj_length=MAXIMUM_TRAJECTORY_LENGTH)
    end = timer()
    print(f"total time Generating trajectories for the candidate policy... {end - start}")
    
    start = timer()
    print("Computing value estimates for candidate trajectories...")
    candidate_value_estimates.append(irl_agent.compute_value_estimate(trajs=candidate_trajectories))
    end = timer()
    print(f"total time Computing value estimates for candidate trajectories... {end - start}")

    start = timer()
    # step 4: obtain new alphas
    print("Solving linear programming...")
    irl_agent.solve_lp(optimal_value_estimate, candidate_value_estimates)
    end = timer()
    print(f"total time Solving linear programming... {end - start}")    

    # step 5: construct new reward function from the alphas
    start = timer()
    reward_func = irl_agent.construct_reward_function(alphas=irl_agent.get_alphas())
    end = timer()
    print(f"total time construct new reward function from the alphas {end - start}")
    
    # step 6: find optimal policy under new reward function and add to 'candidate_policies' list
    start = timer()
    environment.set_reward_func(reward_func)
    candidate_policies.append(train_func(gw_env=environment, verbose=False))  # train_value_iteration(gw_env=environment))
    # store new reward function
    reward_func_estimates.append(environment.get_board())
    end = timer()
    print(f"total time find optimal policy under new reward function {end - start}")

    print("Latest estimated reward function:\n", reward_func_estimates[-1])
    environment.display_policy(policy=candidate_policies[-1])
    print("============================================================\n" * 2)

    
#return {'reference_reward_func': reward_func_ref, 'policy_pred': np.mean(np.array([list(pol.values()) for pol in candidate_policies]), axis=0), 'avg_predicted_reward_func': np.mean(np.array(reward_func_estimates), axis=0)}

Iteration 0...
Generating trajectories for the candidate policy...
total time Generating trajectories for the candidate policy... 0.03631150000001071
Computing value estimates for candidate trajectories...
total time Computing value estimates for candidate trajectories... 173.44049070000074
Solving linear programming...
total time Solving linear programming... 0.0015633000002708286
total time construct new reward function from the alphas 0.27644750000035856
total time find optimal policy under new reward function 0.004434200000105193
Latest estimated reward function:
 [[-2.33182064 -2.65797864 -2.65797864]
 [-2.65797864 -3.02975724 -3.02975724]
 [-2.65797864 -3.02975724 -3.02975724]]
Policy:
[['>' '>' 'x']
 ['^' '^' '^']
 ['^' '^' '^']]

Iteration 1...
Generating trajectories for the candidate policy...
total time Generating trajectories for the candidate policy... 0.008505699999659555
Computing value estimates for candidate trajectories...
total time Computing value estimates for cand

In [ ]:
def irl_reward_estimation(env: Grid_World, optimal_trajectories: List[List[Any]], train_func: Callable):

    # store reference reward function
    reward_func_ref = deepcopy(env.get_board())
    print('Reference reward function:\n', reward_func_ref)

    irl_agent = IRL_from_sampled_trajectories(d=(GW_SIZE[0] * 4, GW_SIZE[1] * 4),
                                              env_ranges=((0, GW_SIZE[0]), (0, GW_SIZE[1])),
                                              env_discrete_size=GW_SIZE,
                                              penalty_factor=2,
                                              gamma=GAMMA)

    # step 2: given optimal trajectories, compute the value estimate
    print("Computing value estimates for optimal trajectories...")
    optimal_value_estimate = irl_agent.compute_value_estimate(trajs=optimal_trajectories)

    candidate_policies = [env.construct_random_policy()]
    candidate_value_estimates = []
    reward_func_estimates = []

    # while True:
    for i in range(IRL_TRAINING_N):
        print(f"Iteration {i}...")

        # step 3: generate trajectories and compute the value estimate for a random policy
        print("Generating trajectories for the candidate policy...")
        candidate_trajectories = env.generate_trajectories(policy=candidate_policies[-1],
                                                           n_traj=NUMBER_OF_TRAJECTORIES,
                                                           max_traj_length=MAXIMUM_TRAJECTORY_LENGTH)
        print("Computing value estimates for condidate trajectories...")
        candidate_value_estimates.append(irl_agent.compute_value_estimate(trajs=candidate_trajectories))

        # step 4: obtain new alphas
        print("Solving linear programming...")
        irl_agent.solve_lp(optimal_value_estimate, candidate_value_estimates)

        # step 5: construct new reward function from the alphas
        reward_func = irl_agent.construct_reward_function(alphas=irl_agent.get_alphas())

        # step 6: find optimal policy under new reward function and add to 'candidate_policies' list
        env.set_reward_func(reward_func)
        candidate_policies.append(train_func(gw_env=env, verbose=True))  # train_value_iteration(gw_env=env))
        # store new reward function
        reward_func_estimates.append(env.get_board())
        
        print("Latest estimated reward function:\n", reward_func_estimates[-1])
        env.display_policy(policy=candidate_policies[-1])
        print("============================================================\n" * 2)

    return {'reference_reward_func': reward_func_ref, 'policy_pred': np.mean(np.array([list(pol.values()) for pol in candidate_policies]), axis=0), 'avg_predicted_reward_func': np.mean(np.array(reward_func_estimates), axis=0)}


def calc_value_distance(value_estimates_ref, value_estimates_pred):
    return np.linalg.norm(np.array(value_estimates_ref) - np.array(value_estimates_pred))

## Bayesian Inverse Reinforcement Learning (BIRL)

### Hands On - BIRL

In [176]:
def train_value_iteration_new(gw_env: Grid_World, 
                          policy: Dict[Any, Any] = None,
                          value_function: Dict[Any, float] = None,
                          verbose=False,
                          gamma=0.95,
                          training_iterations=1000
                         ) -> Tuple[Dict[Any, Any], Dict[Any, float]]:
    
    vi_agent = ValueIterationAgent(states=gw_env.get_state_space(),
                                   terminal_states=gw_env.get_terminal_states(),
                                   reward_function=gw_env.get_reward_func(),
                                   actions=gw_env.get_action_space(),
                                   gamma=gamma)

    if policy and value_function:
        raise ValueError("Can't pass both policy and value_function arguments at the same time.")
    
    if policy:
        if verbose:
            print("Calculating value function of the given policy via value evaluation...")
        vi_agent.set_value_function(new_value_function=perform_value_evaluation(gw_env, policy, verbose))
        
    if value_function:
        if verbose:
            print("Using the given value function...")
        vi_agent.set_value_function(new_value_function=value_function)
    
    iters = 0
    while iters < training_iterations and not vi_agent.value_converged:

        for state in gw_env.get_state_space():

            if state in gw_env.get_terminal_states():
                continue

            opt_act = vi_agent.get_optimal_action(action_state_pairs=gw_env.get_action_state_pairs(state=state))
            next_state = gw_env.get_new_state_on_action(old_state=state, action=opt_act)
            next_state_value = vi_agent.get_state_value(state=next_state)

            vi_agent.set_state_value(state=state, new_value=(gw_env.get_state_reward(state=next_state) + gamma * next_state_value))

        iters += 1

    if verbose:
        gw_env.display_value_function(value_func=vi_agent.get_value_function())

    vi_agent.construct_greedy_policy(gw_env.get_action_state_pairs)

    if verbose:
        gw_env.display_policy(policy=vi_agent.get_policy())
        
    return vi_agent.get_policy(), vi_agent.get_value_function()


#### Reward Space functionality

In [177]:
def get_random_reward(size: Tuple[int], r_max: float = 1.0, rounding: int = 3) -> np.ndarray:
        
    return np.random.uniform(low=-abs(r_max), high=abs(r_max), size=size).round(rounding)

def get_reward_neighbour(reward: np.ndarray, step_size: float, r_max: float = 1.0) -> np.ndarray:
    
    movement = np.random.randint(low=-1, high=2, size=reward.shape) # random array of -1, 0 and +1
    
    return (reward + step_size * movement).clip(min=-abs(r_max), max=abs(r_max)) # new neighbour reward

In [178]:
get_reward_neighbour(np.array([1, 1, 1, 1, 1, 1]), 0.2)

array([1. , 0.8, 1. , 0.8, 1. , 0.8])

#### Prior, Evidence and Posterior Distribution

In [179]:
def improper_prior(reward):
    return 1

def compute_log_posterior(gw_env:Grid_World,
                          observations: List[List[Tuple[Any]]],
                          reward: np.ndarray,
                          policy: Dict[Any, Any],
                          prior: Callable,
                          alpha: float,
                          gamma: float
                         ) -> float:
    
    gw_env.set_board(new_board=reward)
    q = perform_q_function_evaluation(gw_env=gw_env, policy=policy)
    
    log_p = 0
    
    for observation in observations:
        log_p += np.sum([alpha * q[s][a] - np.log(np.sum(np.exp(alpha * np.array(list(q[s].values()))))) for s, a in observation])
        #### Should we use np.mean() here? our obs are not all of the same length
        # log_p += np.mean([alpha * q[s][a] - np.log(np.sum(np.exp(alpha * np.array(list(q[s].values()))))) for s, a in observation])
    
    log_p += np.log(prior(reward))
    
    return log_p

def posterior_dist_ratio(gw_env: Grid_World,
                         observations: List[List[Tuple[Any]]],
                         reward_next: np.ndarray,
                         policy_next: Dict[Any, Any],
                         reward_curr: np.ndarray,
                         policy_curr: Dict[Any, Any]
                        ) -> float:
    
    prior = improper_prior
    alpha = 10
    gamma = GAMMA
    
    log_post_next = compute_log_posterior(gw_env, observations, reward_next, policy_next, prior, alpha, gamma)
    log_post_curr = compute_log_posterior(gw_env, observations, reward_curr, policy_curr, prior, alpha, gamma)
    
    return np.exp(log_post_next - log_post_curr)


#### PolicyWalk sampling algorithm

Credits for helping to decypher the formulation of the paper go to...

- https://github.com/uidilr/bayesian_irl

In [180]:
def policyWalk(observations: List[List[Tuple[Any]]],
               gw_env: Grid_World,
               step_size: float,
               n_steps: int,
               samples_burn_in: int = 500,
               samples_n_out: int = 5
              ) -> np.ndarray:
    
    # step 1: pick a random reward from the reward space (as np.array)
    reward_0 = get_random_reward(size=gw_env.get_board().shape)
    
    # step 2: get optimal policy given reward_0
    gw_env.set_board(new_board=reward_0)  # board is the np.array representation of the reward function
    policy_0, val_func_0 = train_value_iteration_new(gw_env=gw_env)
    
    posterior_samples = []
    
    # step 3: explore the reward space and sample from the posterior (GridWalk as a MCMC (Metropolis algorithm))
    for i in range(n_steps):
        # step 3a: pick reward_1 from reward_0 neighbours randomly (step_size distance)
        reward_1 = get_reward_neighbour(reward=reward_0, step_size=step_size)
        
        # step 3b: compute the Q function of reward_1 under policy_0
        gw_env.set_board(new_board=reward_1)
        q_func = perform_q_function_evaluation(gw_env=gw_env,
                                               policy=policy_0)
        
        # step 3c: check if ... 
        if is_policy_not_optimal(q_func=q_func, policy=policy_0,
                                 states=gw_env.get_state_space(), actions=gw_env.get_action_space()):
            # step 3c.i: get optimal policy given reward_1 AND policy_0 (we can use the value function of policy_0 directly)
            #### MOTIVATION: since reward_1 is a neighbour of reward_0, policy_1 will be close to policy_0. We use this fact to speed up the computation
            #### would be a repetition of step 3b: gw_env.set_board(new_board=reward_1)
            policy_1, val_func_1 = train_value_iteration_new(gw_env=gw_env,
                                                         value_function=val_func_0)
            
            # step 3c.ii: update reward_0 AND policy_0 with prob. according to the posterior_dist
            if np.random.uniform() < np.minimum(1, posterior_dist_ratio(gw_env, observations, reward_1, policy_1, reward_0, policy_0)):
                reward_0 = deepcopy(reward_1)
                policy_0 = deepcopy(policy_1)
                val_func_0 = deepcopy(val_func_1)
            
        else:
            # step 3c.iii: update reward_0 with prob. according to the posterior_dist
            if np.random.uniform() < np.minimum(1, posterior_dist_ratio(gw_env, observations, reward_1, policy_0, reward_0, policy_0)):
                reward_0 = deepcopy(reward_1)
                
        if (i > samples_burn_in) and (samples_n_out != 0) and ((i - samples_burn_in) % samples_n_out == 0):
            posterior_samples.append(reward_0)

    # step 4: return a sample from the posterior
    if samples_n_out != 0:
        return posterior_samples
    else:
        return reward_0

def is_policy_not_optimal(q_func, policy, states, actions):
    for state in states:
        for action in actions:
            if q_func[state][policy[state]] < q_func[state][action]:
                return True
    return False


In [201]:
GAMMA = 0.95

VALUE_ITERATION_TRAINING_N = 1000

GW_SIZE = (3, 3)
GW_TRAPS = []
GW_GOALS = [(0, 0)]

NUMBER_OF_STATES = GW_SIZE[0] * GW_SIZE[1]

NUMBER_OF_TRAJECTORIES    = NUMBER_OF_STATES * 20
MAXIMUM_TRAJECTORY_LENGTH = NUMBER_OF_STATES * 4

BIRL_N_OF_POSTERIOR_SAMPLES     = 2
BIRL_POLICYWALK_STEP_SIZE       = 0.05
BIRL_POLICYWALK_N_STEPS         = int(NUMBER_OF_STATES / BIRL_POLICYWALK_STEP_SIZE * 20)
BIRL_POLICYWALK_SAMPLES_BURN_IN = int(BIRL_POLICYWALK_N_STEPS * 0.7)
BIRL_POLICYWALK_SAMPLES_N_OUT   = 10


In [182]:
environment = Grid_World(size=GW_SIZE, traps=GW_TRAPS, goals=GW_GOALS, randomize_board=False)

target_reward = environment.get_board()

vi_greedy_policy, vi_greedy_value_function = train_value_iteration_new(gw_env=environment, verbose=True, gamma=GAMMA, training_iterations=VALUE_ITERATION_TRAINING_N)

print(f"Generating {NUMBER_OF_TRAJECTORIES} trajectories...")

trajectories = environment.generate_trajectories(policy=vi_greedy_policy,
                                                 n_traj=NUMBER_OF_TRAJECTORIES,
                                                 max_traj_length=MAXIMUM_TRAJECTORY_LENGTH,
                                                 return_state_action_pairs=True)


Value function:
[[0.       1.       0.95    ]
 [1.       0.95     0.9025  ]
 [0.95     0.9025   0.857375]]
Policy:
[['x' '<' '<']
 ['^' '<' '<']
 ['^' '<' '<']]
Generating 180 trajectories...


### Manually go through one iteration of policy walk

In [197]:
print(f"Generate 1 sample from the posterior")
print()

posterior_samples = []

environment = Grid_World(size=GW_SIZE, traps=GW_TRAPS, goals=GW_GOALS, randomize_board=False)
policy_opt, val_func_opt = train_value_iteration_new(gw_env=environment)

# step 1: pick a random reward from the reward space (as np.array)
reward_0 = get_random_reward(size=environment.get_board().shape)

print("target reward: ")
print(environment.get_board())
print()

print("optimal policy:")
print(get_display_policy(policy = policy_opt, actions_to_str_map=actions_to_str_map, size=GW_SIZE, terminal_states=GW_GOALS))
print()

print("randomly initialised reward: ")
print(reward_0)

Generate 1 sample from the posterior

target reward: 
[[1. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]

optimal policy:
[['x' '<' '<']
 ['^' '<' '<']
 ['^' '<' '<']]

randomly initialised reward: 
[[-0.354 -0.902 -0.642]
 [-0.824  0.186  0.865]
 [-0.78  -0.738 -0.741]]


In [198]:
# step 2: get optimal policy given reward_0
environment.set_board(new_board=reward_0)  # board is the np.array representation of the reward function
policy_0, val_func_0 = train_value_iteration_new(gw_env=environment)

print("new policy generated by new reward function: ")
print(get_display_policy(policy = policy_0, actions_to_str_map=actions_to_str_map, size=GW_SIZE, terminal_states=GW_GOALS))

print()
print("new value function generated by new reward function and policy")
print(get_display_value_func(val_func_0, size=GW_SIZE, terminal_states=GW_GOALS))

new policy generated by new reward function: 
[['x' 'v' 'v']
 ['>' '>' '<']
 ['>' '^' '^']]

new value function generated by new reward function and policy
[[ 0.         10.33420948 10.68249901]
 [10.33420948 10.68249901 10.33437406]
 [ 9.07949901 10.33437406 10.68265535]]


In [200]:
posterior_samples_history = []

observations=trajectories
gw_env=environment
step_size=BIRL_POLICYWALK_STEP_SIZE
n_steps=BIRL_POLICYWALK_N_STEPS
samples_burn_in=BIRL_POLICYWALK_SAMPLES_BURN_IN
samples_n_out=BIRL_POLICYWALK_SAMPLES_N_OUT

# step 3: explore the reward space and sample from the posterior (GridWalk as a MCMC (Metropolis algorithm))
for i in range(BIRL_POLICYWALK_N_STEPS):
    # step 3a: pick reward_1 from reward_0 neighbours randomly (step_size distance)
    reward_1 = get_reward_neighbour(reward=reward_0, step_size=BIRL_POLICYWALK_STEP_SIZE)

    # step 3b: compute the Q function of reward_1 under policy_0
    environment.set_board(new_board=reward_1)
    q_func = perform_q_function_evaluation(gw_env=environment,
                                           policy=policy_0)

    # step 3c: check if ... 
    if is_policy_not_optimal(q_func=q_func, policy=policy_0,
                             states=environment.get_state_space(), actions=environment.get_action_space()):
        # step 3c.i: get optimal policy given reward_1 AND policy_0 (we can use the value function of policy_0 directly)
        #### MOTIVATION: since reward_1 is a neighbour of reward_0, policy_1 will be close to policy_0. We use this fact to speed up the computation
        #### would be a repetition of step 3b: gw_env.set_board(new_board=reward_1)
        policy_1, val_func_1 = train_value_iteration_new(gw_env=environment,
                                                     value_function=val_func_0)

        # step 3c.ii: update reward_0 AND policy_0 with prob. according to the posterior_dist
        if np.random.uniform() < np.minimum(1, posterior_dist_ratio(environment, trajectories, reward_1, policy_1, reward_0, policy_0)):
            reward_0 = deepcopy(reward_1)
            policy_0 = deepcopy(policy_1)
            val_func_0 = deepcopy(val_func_1)

    else:
        # step 3c.iii: update reward_0 with prob. according to the posterior_dist
        if np.random.uniform() < np.minimum(1, posterior_dist_ratio(environment, trajectories, reward_1, policy_0, reward_0, policy_0)):
            reward_0 = deepcopy(reward_1)

    if (i > BIRL_POLICYWALK_SAMPLES_BURN_IN) and (BIRL_POLICYWALK_SAMPLES_N_OUT != 0) and ((i - BIRL_POLICYWALK_SAMPLES_BURN_IN) % BIRL_POLICYWALK_SAMPLES_N_OUT == 0):
        posterior_samples.append(reward_0)

# step 4: return a sample from the posterior
if BIRL_POLICYWALK_SAMPLES_N_OUT != 0:
    posterior_samples_history += posterior_samples
else:
    posterior_samples_history += reward_0

#### Run multiple iterations for posterior sampling

In [162]:
print(f"Generate {BIRL_N_OF_POSTERIOR_SAMPLES} samples from the posterior")

posterior_samples = []

for _ in tqdm(range(BIRL_N_OF_POSTERIOR_SAMPLES)):
    
    # restart the environment
    environment = Grid_World(size=GW_SIZE, traps=GW_TRAPS, goals=GW_GOALS, randomize_board=False)

    posterior_samples += policyWalk(observations=trajectories,
                                    gw_env=environment,
                                    step_size=BIRL_POLICYWALK_STEP_SIZE,
                                    n_steps=BIRL_POLICYWALK_N_STEPS,
                                    samples_burn_in=BIRL_POLICYWALK_SAMPLES_BURN_IN,
                                    samples_n_out=BIRL_POLICYWALK_SAMPLES_N_OUT)

Generate 10 samples from the posterior


  0%|                                                                                           | 0/10 [00:00<?, ?it/s]C:\Users\NiklasKaspareit\AppData\Local\Temp\ipykernel_20360\2626929354.py:42: RuntimeWarning: overflow encountered in exp
  return np.exp(log_post_next - log_post_curr)
100%|█████████████████████████████████████████████████████████████████████████████████| 10/10 [19:40<00:00, 118.02s/it]
